In [1]:
#Basic imports
import pandas as pd 
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Modelling
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso 
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [2]:
df=pd.read_csv('data/StudentsPerformance.csv')

In [3]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
# Preparing X and Y variable
x=df.drop(columns=['math score'],axis=1)

In [5]:
y=df['math score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [6]:
# create column transformer with 3 types of transformewrs
num_feature= x.select_dtypes(exclude="object").columns
cat_feature= x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
        [
            ("OneHotEncoder",oh_transformer,cat_feature),
            ("StandardScaler",numeric_transformer,num_feature),
            
        ]
)



In [7]:
x=preprocessor.fit_transform(x)

In [8]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [9]:
x.shape

(1000, 19)

In [13]:
# separate the dataset into train and split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((800, 19), (200, 19))

Create Evaluate function ti give all matrices after model trsiinig

In [14]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_absolute_error(true,predicted))
    r2_score = r2_score(true,predicted)
    return mae,mse,rmse,r2_score

In [22]:
models = {
        "Linear Regression": LinearRegression(),
        "Lasso": Lasso(),
        "Ridge": Ridge(),
        "K-Neighbors Regressor": KNeighborsRegressor(),
        "Decision Tree":DecisionTreeRegressor(),
        "Random Forest Regressor":RandomForestRegressor(),
        "XGBRegressor":XGBRegressor(),
        "CatBoosting Regressor":CatBoostRegressor(verbose=0),
        "AdaBoost Regressor":AdaBoostRegressor()
}   
model_list =[]
r2_list =[]

for i in range(len(list(models))):
    model =list(models.values())[i]
    model.fit(x_train,y_train)

    # make predictions
    y_train_pred = model.predict(x_train)
    y_test_pred=model.predict(x_test)


 #evaluate Train and test dataset
    model_train_mae,model_train_rmse,model_train_r2 = evaluate_model(y_test,y_test_pred)
    model_test_mae,model_test_rmse,model_test_r2 = evaluate_model(y_test,y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

print(" model performance for Training set")
print("Root Mean Square Error: {:,4f}".format(model_train_rmse))
print("= Mean Absolute Error: {:,4f}".format(model_train_mae))
print("= R2 score :{:,4f}".format(model_train_r2))
print("----------")

print("model performance for testing set")

print("Root mean squared error :{:4f}".format(model_test_rmse))
print(" mean absolute squared error :{:4f}".format(model_test_mae))
print("r2 score :{:4f}".format(model_test_r2))


r2_list.append(model_train_r2)


print('='*35)
print('\n')


UnboundLocalError: cannot access local variable 'r2_score' where it is not associated with a value

Results


In [20]:
pd.DataFrame(list(zip(model_list,r2_list),columns=['Model Name','R2_score']).sort_values(by="[r2_score]",ascending=False))

TypeError: list() takes no keyword arguments